In [26]:
import os
import shutil
import certifi
import tempfile

caminho_ruim_cert = certifi.where()
print(f"Original (com erro): {caminho_ruim_cert}")

with tempfile.NamedTemporaryFile(delete=False, suffix='.pem') as tmp_file:
    with open(caminho_ruim_cert, 'rb') as source_file:
        shutil.copyfileobj(source_file, tmp_file)
    caminho_seguro_cert = tmp_file.name

print(f"Novo caminho seguro criado: {caminho_seguro_cert}")

os.environ['CURL_CA_BUNDLE'] = caminho_seguro_cert
os.environ['SSL_CERT_FILE'] = caminho_seguro_cert
os.environ['REQUESTS_CA_BUNDLE'] = caminho_seguro_cert

import yfinance as yf
import polars as pl
import polars.selectors as cs
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

Original (com erro): c:\Users\william.mendes\OneDrive\Área de Trabalho\projects\pos\tech_challenge_2_mlet\.venv\Lib\site-packages\certifi\cacert.pem
Novo caminho seguro criado: C:\Users\WILLIA~1.MEN\AppData\Local\Temp\tmpqj0tn665.pem


In [27]:
# TOTS3: Software (ERP) - Líder de mercado
# LWSA3: Infraestrutura de internet/Hospedagem
# POSI3: Hardware/Tecnologia (Positivo)
# INTB3: Segurança/Redes (Intelbras)
# WEGE3: Automação Industrial/Energia (WEG)
tickers = ['TOTS3.SA', 'LWSA3.SA', 'POSI3.SA', 'INTB3.SA', 'WEGE3.SA']

In [28]:
today = datetime.now()

# D-1 
end_date_obj = today - timedelta(days=1)
end_date = end_date_obj.strftime('%Y-%m-%d')

# Dados dos últimos 6 meses
start_date_obj = end_date_obj - relativedelta(months=6)
start_date = start_date_obj.strftime('%Y-%m-%d')

print(f"Executando pipeline para janela: {start_date} até {end_date}")

Executando pipeline para janela: 2025-07-10 até 2026-01-10


In [ ]:
data_list = []

print("\n >>> Iniciando Download das ações")

for ticker in tickers:
    try:
        df_y = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if df_y.empty:
            print(f"Vazio: {ticker}")
            continue

        # Normalização para Polars
        df_y = df_y.reset_index()
        df_y['Ticker'] = ticker
        
        # Reseta o índice caso seja MultiIndex
        if isinstance(df_y.columns, pd.MultiIndex):
            df_y.columns = df_y.columns.get_level_values(0)

        # Converte para Polars
        pl_df = pl.from_pandas(df_y)
        data_list.append(pl_df)
        print(f"Sucesso: {ticker}")
        
    except Exception as e:
        print(f"Erro {ticker}: {e}")

if data_list:
    df_raw = pl.concat(data_list)
    df_raw = df_raw.sort(["Ticker", "Date"])
    print(f"\nTotal de linhas baixadas: {df_raw.shape[0]}")
    print(df_raw.head())


--- Iniciando Download das ações ---
Sucesso: TOTS3.SA
Sucesso: LWSA3.SA
Sucesso: POSI3.SA
Sucesso: INTB3.SA
Sucesso: WEGE3.SA

Total de linhas baixadas: 635
shape: (5, 7)
┌─────────────────────┬───────────┬───────────┬───────────┬───────────┬─────────┬──────────┐
│ Date                ┆ Close     ┆ High      ┆ Low       ┆ Open      ┆ Volume  ┆ Ticker   │
│ ---                 ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---     ┆ ---      │
│ datetime[ns]        ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ i64     ┆ str      │
╞═════════════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═════════╪══════════╡
│ 2025-07-10 00:00:00 ┆ 13.171747 ┆ 13.281817 ┆ 12.979124 ┆ 13.199265 ┆ 1923700 ┆ INTB3.SA │
│ 2025-07-11 00:00:00 ┆ 13.098367 ┆ 13.21761  ┆ 12.942434 ┆ 13.171747 ┆ 1575000 ┆ INTB3.SA │
│ 2025-07-14 00:00:00 ┆ 13.263472 ┆ 13.410232 ┆ 12.804845 ┆ 12.933261 ┆ 1925600 ┆ INTB3.SA │
│ 2025-07-15 00:00:00 ┆ 13.777134 ┆ 13.896376 ┆ 13.217609 ┆ 13.217609 ┆ 1619300 ┆ I

In [30]:
# Transformação (Feature Engineering)
df_refined = df_raw.with_columns([
    # Cast de data
    pl.col("Date").cast(pl.Date).alias("data_pregao"),

    # Lowercase
    pl.col("Ticker").str.to_lowercase().alias("nome_acao"),

    # Renomeando colunas
    pl.col("Open").alias("abertura"),
    pl.col("Close").alias("fechamento"),
    pl.col("High").alias("max"),
    pl.col("Low").alias("min"),
    pl.col("Volume").alias("volume_negociado"),
    
    # Média Móvel de 7 dias agrupado por Ticker
    pl.col("Close")
        .rolling_mean(window_size=7)
        .over("Ticker")
        .alias("media_movel_7d"),

    # Lag 1: O valor de fechamento do dia anterior agrupado por Ticker
    pl.col("Close")
        .shift(1)
        .over("Ticker")
        .alias("lag_1d"),
        
    # Lag 2: O valor de fechamento de 2 dias atrás agrupado por Ticker
    pl.col("Close")
        .shift(2)
        .over("Ticker")
        .alias("lag_2d"),
    
    # Lag 3: O valor de fechamento de 3 dias atrás agrupado por Ticker
    pl.col("Close")
        .shift(3)
        .over("Ticker")
        .alias("lag_3d")
])

# Remove linhas com valores nulos
df_final = df_refined.drop_nulls()

# Arredonda valores float para 2 casas decimais
df_final = df_final.with_columns(cs.float().round(2))

# Remove colunas desnecessárias
df_final = df_final.select([
    "data_pregao", "nome_acao", "abertura", "fechamento", "max", "min",
    "volume_negociado", "media_movel_7d", "lag_1d", "lag_2d", "lag_3d"
])

In [ ]:
print(df_final.head(10))


--- Amostra com Lags e Média Móvel ---
shape: (10, 11)
┌─────────────┬───────────┬──────────┬────────────┬───┬────────────────┬────────┬────────┬────────┐
│ data_pregao ┆ nome_acao ┆ abertura ┆ fechamento ┆ … ┆ media_movel_7d ┆ lag_1d ┆ lag_2d ┆ lag_3d │
│ ---         ┆ ---       ┆ ---      ┆ ---        ┆   ┆ ---            ┆ ---    ┆ ---    ┆ ---    │
│ date        ┆ str       ┆ f64      ┆ f64        ┆   ┆ f64            ┆ f64    ┆ f64    ┆ f64    │
╞═════════════╪═══════════╪══════════╪════════════╪═══╪════════════════╪════════╪════════╪════════╡
│ 2025-07-18  ┆ intb3.sa  ┆ 13.76    ┆ 13.35      ┆ … ┆ 13.47          ┆ 13.88  ┆ 13.79  ┆ 13.78  │
│ 2025-07-21  ┆ intb3.sa  ┆ 13.4     ┆ 13.62      ┆ … ┆ 13.54          ┆ 13.35  ┆ 13.88  ┆ 13.79  │
│ 2025-07-22  ┆ intb3.sa  ┆ 13.61    ┆ 13.35      ┆ … ┆ 13.57          ┆ 13.62  ┆ 13.35  ┆ 13.88  │
│ 2025-07-23  ┆ intb3.sa  ┆ 13.35    ┆ 13.72      ┆ … ┆ 13.64          ┆ 13.35  ┆ 13.62  ┆ 13.35  │
│ 2025-07-24  ┆ intb3.sa  ┆ 13.72    ┆ 13.65

In [ ]:
# Agrupamento/Sumarização
df_agregado = df_refined.group_by(["Ticker", pl.col("data_pregao").dt.truncate("1mo")]).agg([
    pl.col("volume_negociado").sum().alias("volume_mensal_total"),
    pl.col("fechamento").mean().alias("preco_medio_mensal").round(2)
]).sort(["Ticker", "data_pregao"])

print("\n--- Agregação Mensal ---")
print(df_agregado.head())


--- Agregação Mensal ---
shape: (5, 4)
┌──────────┬─────────────┬─────────────────────┬────────────────────┐
│ Ticker   ┆ data_pregao ┆ volume_mensal_total ┆ preco_medio_mensal │
│ ---      ┆ ---         ┆ ---                 ┆ ---                │
│ str      ┆ date        ┆ i64                 ┆ f64                │
╞══════════╪═════════════╪═════════════════════╪════════════════════╡
│ INTB3.SA ┆ 2025-07-01  ┆ 32258900            ┆ 13.54              │
│ INTB3.SA ┆ 2025-08-01  ┆ 41764600            ┆ 11.8               │
│ INTB3.SA ┆ 2025-09-01  ┆ 49118000            ┆ 11.68              │
│ INTB3.SA ┆ 2025-10-01  ┆ 52991000            ┆ 10.65              │
│ INTB3.SA ┆ 2025-11-01  ┆ 38471300            ┆ 11.7               │
└──────────┴─────────────┴─────────────────────┴────────────────────┘


In [33]:
# Exemplo de salvamento local em Parquet (como seria feito no S3)
# df_refined.write_parquet("dados_refined.parquet")